In [19]:
import Pkg; Pkg.add("BenchmarkTools")

  Installing known registries into `~/.julia`


    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


   Installed Parsers ───────── v2.5.0
   Installed BenchmarkTools ── v1.3.2
   Installed SnoopPrecompile ─ v1.0.1
   Installed JSON ──────────── v0.21.3


    Updating `~/.julia/environments/v1.8/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2
  [682c06a0] + JSON v0.21.3
  [69de0a69] + Parsers v2.5.0
  [66db9d55] + SnoopPrecompile v1.0.1
  [56f22d72] + Artifacts
  [ade2ca70] + Dates
  [8f399da3] + Libdl
  [37e2e46d] + LinearAlgebra
  [56ddb016] + Logging
  [a63ad114] + Mmap
  [de0858da] + Printf
  [9abbd945] + Profile
  [9a3f8284] + Random
  [ea8e919c] + SHA v0.7.0
  [9e88b42a] + Serialization
  [2f01184e] + SparseArrays
  [10745b16] + Statistics
  [cf7118a7] + UUIDs
  [4ec0a83e] + Unicode
  [e66e0078] + CompilerSupportLibraries_jll v0.5.2+0
  [4536629a] + OpenBLAS_jll v0.3.20+0
  [8e850b90] + libblastrampoline_jll v5.1.1+0


...

  ✓ libblastrampoline_jll


  ✓ Parsers


  ✓ JSON


  ✓ BenchmarkTools


  7 dependencies successfully precompiled in 8 seconds


In [3]:
using BenchmarkTools

## Collatz conjecture
Given a positive integer, create a function that counts the steps it takes to reach `1` following the [Collatz conjecture algorithm](https://en.wikipedia.org/wiki/Collatz_conjecture) (if $n$ is odd do $n=3n+1$ otherwise do $n=n/2$). Test it with the number 100 to get 25.

*Hint: make a type-stable function by using `÷`, (`\div<TAB>`): In Julia `/` is the floating point devision operator and thus `n/m` is always a float number even if `n, m` are integers.*

In [34]:
function collatz(n)
    n==1 ? 0 : isodd(n) ? 1+collatz(3n+1) : 1+collatz(n÷2)
end

collatz (generic function with 1 method)

In [39]:
@btime collatz(100)

  32.110 ns (0 allocations: 0 bytes)


25

In [38]:
@btime collatz(989345275647)

  8.917 μs (0 allocations: 0 bytes)


1348

## Fibonacci numbers
Using recursion (a function that calls itself) create a function that given `n` it returns the `n`-th [Fibonacci number](https://en.wikipedia.org/wiki/Fibonacci_number). Apply it using broadcast to `1:8` to get the result `[1,1,2,3,5,8,13]`.

In [1]:
function fibo(n)
    n ∈ (1,2) ? 1 : fibo(n-2) + fibo(n-1)
end

fibo (generic function with 1 method)

In [4]:
@btime fibo.(1:8)

  136.863 ns (1 allocation: 128 bytes)


8-element Vector{Int64}:
  1
  1
  2
  3
  5
  8
 13
 21

## Logistic map
Create a function that given `r, x0, N` it creates an orbit of length `N` of the logistic map $x_{n+1} = f(x) = rx(1-x)$. This map produces an orbit iteratively, i.e. starting from `x0` one applies `f(x0)` to get `x1`, and then applies `f(x1)` to get `x2`, and so on. 

*Hint: use `zeros` or `fill` to initialize a vector, instead of starting with an empty one and using `push!`*.

In [21]:
logistic(x,r) = r*x*(1-x)
function logistic(x0,r,N)
    re = fill(x0, N)
    for i=2:N
        re[i] = logistic(re[i-1], r)
    end
    return re
end

logistic (generic function with 2 methods)

In [23]:
@btime logistic(0.5, 3.7, 100)

  242.309 ns (1 allocation: 896 bytes)


100-element Vector{Float64}:
 0.5
 0.925
 0.25668749999999985
 0.7059564011718747
 0.7680532550204203
 0.6591455741499428
 0.8312889390453947
 0.518916263804854
 0.9236760473655612
 0.26084484548817055
 ⋮
 0.9039029702319807
 0.3213908453597996
 0.8069654485512382
 0.5763571895642274
 0.9034274445268548
 0.32281129890936666
 0.8088354075615125
 0.5720965568343636
 0.9057677200227289

## Counting nucleotides
Create a function that given a DNA strand (as a `String`, e.g. `"AGAGAGATCCCTTA"`) it counts how much of each nucleotide (A G T or C) is present in the strand. The function should throw an error (using `error`) if an invalid nucleotide is encountered. Test your result with `"ATATATAGGCCAX"` and `"ATATATAGGCCAA"`.

*Hint: there are two ways to solve this one: either making a dictionary and using `haskey` or using `count` in combination with `occursin`.*

In [50]:
function nucleotides_count(strand)
    invalid = setdiff(Set(strand), Set("ATGC"))
    length(invalid) > 0 ? error("Invalid nucleotide encountered: $(collect(invalid))") :
    (
        "A" => count(occursin("A"), strand),
        "B" => count(occursin("B"), strand),
        "C" => count(occursin("C"), strand),
        "D" => count(occursin("D"), strand)
    )
end

nucleotides_count (generic function with 1 method)

In [52]:
nucleotides_count("ATATATAGGCCAA")

("A" => 6, "B" => 0, "C" => 2, "D" => 0)

In [51]:
# We expect an error here
nucleotides_count("ATATATAGGCCAX")

ErrorException: Invalid nucleotide encountered: ['X']

## Hamming distance
Create a function that calculates the Hamming distance of two equal DNA strands, given as strings. This distance is defind by counting (sequentially) the number of non-equal letters in the two strands, e.g. `"ATA"` and `"ATC"` have distance of 1, while `"ATC"` and `"CAT"` have distance of 3. 

*Hint: this exercise has a one-liner solution, using the `zip` and `count` functions.*

In [59]:
hamming_distance(dna1, dna2) = count(x -> x[1]!=x[2], zip(dna1,dna2)) 

hamming_distance (generic function with 1 method)

In [60]:
hamming_distance("ATC", "CAT")

3

In [61]:
hamming_distance("AAAAAA", "AAATCAGGGG")

2

## Babylonian square root
Babylonians knew how to calculate square roots several millenia ago. To get the square root of $y$ they used the algorithm $x_{n+1} = \frac{1}{2}(x_n + \frac{y}{x_n})$ iteratively starting from some value $x_0$ to converge to $x_n \to \sqrt{y}$ as $n\to \infty$. Implement this algortihm in a function `babylonian(y, ε)` that takes some convergence tolerance `ε`. The function should print the amount of steps it took, as well as the square root value.

In [77]:
babylonian_step(y,xn) = 0.5(xn + y/xn)
function babylonian_square_root(y, ϵ=1e-8)
    steps,x,xn = 0,1.0,Inf
    # Tolerance ϵ is between x_n and x_(n+1)
    while abs(x-xn)>ϵ
        xn=x
        x=babylonian_step(y,xn)
        steps+=1
    end
    println("Number of steps: $steps")
    return x
end

babylonian_square_root (generic function with 2 methods)

In [78]:
babylonian_square_root(365)

Number of steps: 9


19.1049731745428

In [79]:
sqrt(365)

19.1049731745428